In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import scipy.sparse as sparse
import sklearn.metrics as metrics
import sys as sys



/Users/tyyoo/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
testdata = pd.read_csv("Test_Train_Sets/All_test.csv")
traindata = pd.read_csv("Test_Train_Sets/All_train.csv")
valdata = pd.read_csv("Test_Train_Sets/All_validate.csv")

In [3]:
traindata.shape, testdata.shape, valdata.shape

((84387, 18), (9910, 18), (13874, 18))

In [4]:
traindata.head()

,Title,ID,Date,Class,Category,Author,ISBN,Publisher,Pub_Date,Order_Time,Count,Cart,Cart_Date,Device,Address1,Address2,user_purchase_count,book_sell_count
0,데프콘 한미전쟁 세트,168,20141108,주문,국내문학,<김경진> 등저,9.788956e+12,씨앗을뿌리는사람,20000229,20,1,N,NaN,기기PID_PC,인천광역시,계양구,1,1
1,다이버전트,186,20141003,주문,해외문학,<베로니카 로스> 저/<이수현> 역,9.788957e+12,은행나무,20130807,18,1,N,NaN,기기PID_PC,제주특별자치도,서귀포시,1,20
2,"스피라, 세계를 향한 영혼의 승부",188,20140929,주문,자기계발,<김한철> 저,9.788926e+12,랜덤하우스코리아,20100916,19,1,N,NaN,기기PID_PC,인천광역시,강화군,1,2
3,이상한 나라의 앨리스,193,20140910,주문,해외문학,<루이스 캐럴> 저/<김양미> 역/<김민지> 그림,9.788993e+12,인디고(글담),20071220,21,1,N,NaN,기기PID_PC,인천광역시,서구,1,67
4,노인과 바다,217,20140904,주문,해외문학,<어니스트 헤밍웨이> 저/<이종인> 역,9.788933e+12,열린책들,20120210,15,1,N,NaN,기기PID_PC,경기도,수원시 장안구,1,68


In [5]:
print '%d different books and %d different users in traindata' %(np.unique(traindata['Title']).shape[0],np.unique(traindata['ID']).shape[0])
print '%d different books and %d different users in testdata' %(np.unique(testdata['Title']).shape[0],np.unique(testdata['ID']).shape[0])
print '%d different books and %d different users in valdata' %(np.unique(valdata['Title']).shape[0],np.unique(valdata['ID']).shape[0])


19148 different books and 18975 different users in traindata
4877 different books and 1982 different users in testdata
6084 different books and 1982 different users in valdata


In [6]:
#dataframe that contains only useful informations
usefulcol = ['Title','ID','Class','Category','Author','ISBN','Count','Address1','Address2','Publisher']
dftrain = traindata[usefulcol]
dftest = testdata[usefulcol]
dfval = valdata[usefulcol]

In [7]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [8]:
title_train = np.unique(dftrain['Title'].values)
author_train = np.unique(dftrain['Author'].values)
publisher_train = np.unique(dftrain['Publisher'].values)
title_train_dict = {i:t for i,t in enumerate(title_train)}
author_train_dict = {i:t for i,t in enumerate(author_train)}
publisher_train_dict = {i:t for i,t in enumerate(publisher_train)}
print title_train.shape[0],author_train.shape[0],publisher_train.shape[0]

19148 13581 2165


In [9]:
import re
def author_to_list(author):
    #split the list of authors in a single string into a numpy list of authors. remove brackets, '저', '역', '그림', '/', and etc
    #remove translater
    if author != author:
        newauthorlist = []
    else:
        split_list = np.array(['/'])
        mask = np.array([s in author for s in split_list])
        split_list = split_list[mask]
        
        for s in split_list:
            author = re.sub(s,' ..',author)
        authorlist = author.split(' ..')
        authorlist = np.array(filter(None,authorlist))
        
        translatormask = np.array([(' 역' in a) or (' 공역' in a) or 
                                   (' 편역' in a) or (' 감수' in a) or 
                                   (' 옮김' in a) or (' 등역' in a) or
                                   (' 엮음' in a) for a in authorlist])
        authorlist = authorlist[~translatormask]
        
        remove_str_list = np.array(['<','>',' 그림',' 저',' 공저',' 글', ' 등저',' 공편','편'])
        newauthorlist = np.array([])
        for astr in authorlist:
            alist = np.array(astr.split(','))
            for i,a in enumerate(alist):
                for s in remove_str_list:
                    a = re.sub(s,'',a)
                alist.flat[i]=a
            newauthorlist = np.append(newauthorlist,alist)
                
    return newauthorlist       

In [10]:
#test
for i in range(400,500):
    chosen_author = author_train_dict[i]
    print chosen_author
    authorlist = author_to_list(chosen_author)
    for a in authorlist:
        print a

<가토 스가> 저/<김대환> 역
가토 스가
<가토 히데토시> 저/<한혜정> 역
가토 히데토시
<각묵> 저
각묵
<각묵스님> 역
<간다 마사노리> 저/<김경인> 역
간다 마사노리
<간다 마사노리> 저/<이선희> 역
간다 마사노리
<간다 와카>,<닛타 아키쓰구> 공저/<이소담> 역
간다 와카
닛타 아키쓰구
<감산> 저/<오진탁> 역
감산
<감성현> 저
감성현
<감초비> 저
감초비
<감태준> 등저
감태준
<감태준>,<김재홍>,<윤명구>,<이건청> 공저
감태준
김재홍
윤명구
이건청
<강경애>
강경애
<강경희> 저
강경희
<강광석> 등저
강광석
<강구영>  편저
강구영  저
<강규석> 저
강규석
<강규원> 저
강규원
<강규형> 저
강규형
<강금만> 저
강금만
<강금실> 저
강금실
<강기호> 저
강기호
<강길웅> 저
강길웅
<강남구> 강의
강남구 강의
<강남구> 저
강남구
<강대진> 저
강대진
<강대진>,<김상근>,<김석>,<이석재>,<이종숙>,<이진우> 등저
강대진
김상근
김석
이석재
이종숙
이진우
<강동수>,<박향> 등저
강동수
박향
<강동화> 저
강동화
<강명옥> 저
강명옥
<강문정> 저
강문정
<강미라> 저
강미라
<강미은> 저
강미은
<강미현> 저
강미현
<강미현> 편
강미현 
<강민경> 저
강민경
<강민석> 저
강민석
<강민혁> 저
강민혁
<강백수> 저
강백수
<강병석> 저
강병석
<강북삼성병원>,<삼성스포츠단> 공저
강북삼성병원
삼성스포츠단
<강산> 저
강산
<강상구> 저
강상구
<강상중> 저
강상중
<강상중> 저/<노수경> 역
강상중
<강상중> 저/<송태욱> 역
강상중
<강상중> 저/<오근영> 역
강상중
<강상중> 저/<이경덕> 역
강상중
<강석경> 저
강석경
<강석진> 저
강석진
<강선영> 저
강선영
<강성률> 저
강성률
<강성원> 등저
강성원
<강성은> 저
강성은
<강성태> 저
강성태
<강성호> 저
강성호
<강세형> 저
강세형
<강수미> 저
강수미
<강수진> 저
강수진
<강승완>,<김선희>,<김용신>,<마석한>,<성해영>,<안병대>,<유헌식>,<이기봉>

In [11]:
#dataframe of all books (no duplicates)
dfbook = dftrain[['Author','Title','Publisher','ISBN']]
dfbook = dfbook[~dfbook.duplicated()]
dfbook = dfbook.reset_index()
dfbook.drop('index', axis=1, inplace=True)
dfbook

,Author,Title,Publisher,ISBN
0,<김경진> 등저,데프콘 한미전쟁 세트,씨앗을뿌리는사람,9.788956e+12
1,<베로니카 로스> 저/<이수현> 역,다이버전트,은행나무,9.788957e+12
2,<김한철> 저,"스피라, 세계를 향한 영혼의 승부",랜덤하우스코리아,9.788926e+12
3,<루이스 캐럴> 저/<김양미> 역/<김민지> 그림,이상한 나라의 앨리스,인디고(글담),9.788993e+12
4,<어니스트 헤밍웨이> 저/<이종인> 역,노인과 바다,열린책들,9.788933e+12
5,조지프 캠벨,신의 가면 3,까치(까치글방),9.788973e+12
6,조지프 캠벨,신의 가면 2,까치(까치글방),9.788973e+12
7,조지프 캠벨 저/정영목 역,신의 가면 4,까치(까치글방),9.788973e+12
8,<최진석> 저,노자의 목소리로 듣는 도덕경,소나무,9.788971e+12
9,<조지프 캠벨> 저/<노혜숙> 역,"블리스, 내 인생의 신화를 찾아서",아니마,9.788997e+12


In [12]:
author_entire_list = np.array([])
for a in author_train:
    author_entire_list = np.append(author_entire_list,author_to_list(a))
author_entire_list = np.unique(author_entire_list)
author_entire_list.shape

(11918,)

In [13]:
author_entire_list_dict = {i:a for i,a in enumerate(author_entire_list)}
author_entire_list_dict

{0: '',
 1: ' J. \xec\x8a\xa4\xec\xbd\x94\xed\x8a\xb8 \xeb\x93\x80\xeb\xb0\x9c',
 2: ' Keith C. Barton',
 3: ' MARILYN BATES',
 4: ' \xea\xb0\x80\xec\x8b\x9c\xed\x95\x98\xeb\x9d\xbc \xeb\x85\xb8\xeb\xb6\x80\xec\x95\xbc',
 5: ' \xea\xb0\x80\xec\x99\x80\xec\x9d\xb4 \xeb\xa7\x88\xec\x82\xac\xeb\xaf\xb8',
 6: ' \xea\xb3\xb5\xec\xa7\x80\xec\x98\x81',
 7: ' \xea\xb7\xb8\xeb\xa0\x88\xec\x9d\xb4\xec\x97\x84 \xea\xb7\xb8\xeb\xa6\xb0',
 8: ' \xea\xb9\x80\xeb\xb4\x89\xea\xb7\x9c',
 9: ' \xea\xb9\x80\xeb\xb4\x89\xeb\xa0\xac',
 10: ' \xea\xb9\x80\xec\x8a\xb9\xeb\xaf\xbc',
 11: ' \xea\xb9\x80\xec\x98\x81\xec\x9d\xbc',
 12: ' \xea\xb9\x80\xec\xa0\x95\xed\x9d\xac',
 13: ' \xea\xb9\x80\xed\x98\xb8\xec\x9d\xb8',
 14: ' \xeb\x82\xa8\xea\xb8\xb0\xec\x88\x99',
 15: ' \xeb\x82\xa8\xec\x9e\x90\xe3\x80\x8d \xec\xa0\x9c\xec\x9e\x91\xed\x8c\x80',
 16: ' \xeb\x8b\x90 \xea\xb2\x8c\xec\x9d\xb4\xeb\xa8\xbc',
 17: ' \xeb\x8d\xb0\xec\x9d\xb4\xeb\xb9\x84\xeb\x93\x9c \xeb\xb2\x84\xec\x8a\xa4',
 18: ' \xeb\x8d\xb0\xec\x

In [117]:
dfbook.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9, 
            ...
            20100, 20101, 20102, 20103, 20104, 20105, 20106, 20107, 20108, 20109], dtype='int64', length=20110)

In [136]:
XXX = np.array([0,1,0,1])
np.where(XXX==1)[0][0]

1

In [138]:
#Make binary author lists: Nbooks X Nauthors matrix
#Use author_entire_list for index-author matching
Xauthor = np.zeros((dfbook.index.shape[0],author_entire_list.shape[0]))
for ind in dfbook.index:
    alist = author_to_list(dfbook.iloc[ind]['Author'])
    for a in alist:
        aind = np.where(author_entire_list==a)[0][0]
        Xauthor[ind,aind] = 1

In [142]:
np.savetxt('Xauthor.txt',Xauthor)

In [144]:
#Make binary publisher lists: Nbooks X Npublihser matrix
#use publisher_train or publisher_train_dict for index matching
Xpublisher = np.zeros((dfbook.index.shape[0],publisher_train.shape[0]))
for ind in dfbook.index:
    pub = dfbook.iloc[ind]['Publisher']
    pind = np.where(publisher_train==pub)[0][0]
    Xpublisher[ind,pind] = 1

In [145]:
np.savetxt('Xpublisher.txt',Xpublisher)

In [90]:
%%time
#Make similarity matrix for titles: Nbooks X Nbooks matrix
#Use dfbooks for index-title matching
#Nbooks =dfbook.index.shape[0]
Nbooks = 2000
simmat_title = np.zeros((Nbooks,Nbooks))
for i in range(Nbooks):
    simmat_title[i,i] = 1.0
    for j in range(i+1,Nbooks):
        simmat_title[i,j] = similar(dfbook.iloc[i]['Title'],dfbook.iloc[j]['Title'])
        simmat_title[j,i] = simmat_title[i,j]


KeyboardInterrupt: 

In [91]:
simmat_title

array([[ 1.        ,  0.36363636,  0.24657534, ...,  0.22222222,
         0.3       ,  0.22535211],
       [ 0.36363636,  1.        ,  0.13559322, ...,  0.2       ,
         0.30434783,  0.1754386 ],
       [ 0.24657534,  0.13559322,  1.        , ...,  0.17391304,
         0.24      ,  0.23255814],
       ..., 
       [ 0.22222222,  0.2       ,  0.17391304, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.3       ,  0.30434783,  0.24      , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.22535211,  0.1754386 ,  0.23255814, ...,  0.        ,
         0.        ,  0.        ]])

In [51]:
userid_test = np.unique(dftest['ID'])
userid_test
relbooks = np.array([])
for userid in userid_test:
    relbooks = np.append(relbooks,np.unique(dftrain[dftrain['ID']==userid]['ISBN']))
relbooks = np.unique(relbooks)

In [110]:
dftrain[dftrain['ID']==userid_test[9]]

,Title,ID,Class,Category,Author,ISBN,Count,Address1,Address2,Publisher
430,런던 스케치,2252,주문,해외문학,<도리스 레싱> 저/<서숙> 역,9.788937e+12,1,경기도,안산시 단원구,민음사
431,안개,2252,주문,해외문학,<미겔 데 우나무노> 저/<조민현> 역,9.788937e+12,1,경기도,안산시 단원구,민음사
432,데미안,2252,주문,해외문학,<헤르만 헤세> 저/<전영애> 역,9.788937e+12,1,경기도,안산시 단원구,민음사
433,사랑하라 한번도 상처받지 않은 것처럼,2252,주문,국내문학,<류시화> 편,9.788996e+12,1,울산광역시,남구,오래된미래
434,왜 나는 너를 사랑하는가,2252,주문,해외문학,<알랭 드 보통> 저/<정영목> 역,9.788987e+12,1,울산광역시,남구,청미래
435,끌림,2252,주문,국내문학,<이병률> 저,9.788994e+12,1,울산광역시,남구,달
436,보통의 존재,2252,주문,국내문학,<이석원> 저,9.788994e+12,1,울산광역시,남구,달
437,자기만의 방,2252,주문,해외문학,<버지니아 울프> 저/<이미애> 역,9.788937e+12,1,경기도,안산시 단원구,민음사
438,철학 VS 철학,2252,주문,인문,<강신주> 저,9.788977e+12,1,경기도,안산시 단원구,그린비
439,손자병법,2252,주문,인문,<손자> 저/<김원중> 역,9.788994e+12,1,경기도,안산시 단원구,글항아리


In [18]:
simmat_title

array([[ 1.        ,  0.36363636,  0.24657534, ...,  0.22222222,
         0.29166667,  0.0952381 ],
       [ 0.36363636,  1.        ,  0.13559322, ...,  0.25806452,
         0.29411765,  0.21428571],
       [ 0.24657534,  0.13559322,  1.        , ...,  0.4       ,
         0.19047619,  0.24561404],
       ..., 
       [ 0.22222222,  0.25806452,  0.4       , ...,  1.        ,
         0.34285714,  0.20689655],
       [ 0.29166667,  0.29411765,  0.19047619, ...,  0.34285714,
         1.        ,  0.1875    ],
       [ 0.0952381 ,  0.21428571,  0.24561404, ...,  0.20689655,
         0.1875    ,  1.        ]])

In [19]:
np.savetxt('simmat_title_1000.txt',simmat_title)

In [92]:
def generate_author_sim_mat(authors):
    Nauthors = authors.shape[0]
    simmat = np.zeros((Nauthors,Nauthors))
    for i in range(Nauthors):
        simmat[i,i] = 1.0
        for j in range(i+1,Nauthors):
            simmat[i,j] = author_list_simliarity(authors[i],authors[j])
            simmat[j,i] = simmat[i,j]
    
    return simmat
        

In [93]:
def generate_sim_mat(arr):
    N = arr.shape[0]
    simmat = np.zeros((N,N))
    for i in range(N):
        simmat[i,i] = 1.0
        for j in range(i+1,N):
            simmat[i,j] = similar(arr[i],arr[j])
            simmat[j,i] = simmat[i,j]
    
    return simmat

In [94]:
%%time
publisher_sim_mat = generate_sim_mat(publisher_train)

CPU times: user 3min 2s, sys: 3.29 s, total: 3min 6s
Wall time: 3min 9s


In [437]:
def book_similarity_vec(abook,booklist,weightdict={'Author':0.5,'Title':0.3,'Publisher':0.2}):
    #book and booklist are the dictionary or dataframe with column 'Author','Title','Publisher'
    #weightdict is the weight in each similarity to calculate weighted average 
    
    sim_arr = np.zeros(booklist.index.shape[0])
    for i in range(abook.index.shape[0]):
        authors_sim_arr = generate_author_sim_vec(abook['Author'].values[0],booklist['Author'].values)==1.0
        titles_sim_arr = generate_sim_vec(abook['Title'].values[0],booklist['Title'].values)
        publisher_sim_arr = generate_sim_vec(abook['Publisher'].values[0],booklist['Publisher'].values)==1.0
    
        sim_arr = sim_arr + weightdict['Author']*authors_sim_arr+weightdict['Title']*titles_sim_arr+weightdict['Publisher']*publisher_sim_arr
    booklist['similarity'] = sim_arr/booklist.index.shape[0]
    return booklist

In [ ]:
dftrain

In [459]:
author_sim_mat = generate_author_sim_mat(authors)

KeyboardInterrupt: 

In [416]:
dftrain_books = dftrain[['Author','Title','Publisher']]
dftrain_books = dftrain_books[~dftrain_books.duplicated()]
dftrain_books = dftrain_books.reset_index()
dftrain_books

,index,Author,Title,Publisher
0,0,<김경진> 등저,데프콘 한미전쟁 세트,씨앗을뿌리는사람
1,1,<베로니카 로스> 저/<이수현> 역,다이버전트,은행나무
2,2,<김한철> 저,"스피라, 세계를 향한 영혼의 승부",랜덤하우스코리아
3,3,<루이스 캐럴> 저/<김양미> 역/<김민지> 그림,이상한 나라의 앨리스,인디고(글담)
4,4,<어니스트 헤밍웨이> 저/<이종인> 역,노인과 바다,열린책들
5,5,조지프 캠벨,신의 가면 3,까치(까치글방)
6,6,조지프 캠벨,신의 가면 2,까치(까치글방)
7,7,조지프 캠벨 저/정영목 역,신의 가면 4,까치(까치글방)
8,8,<최진석> 저,노자의 목소리로 듣는 도덕경,소나무
9,9,<조지프 캠벨> 저/<노혜숙> 역,"블리스, 내 인생의 신화를 찾아서",아니마


In [457]:
A = np.zeros(4)
B = np.ones(4)
np.vstack((A,B))

array([[ 0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.]])

In [436]:
users = np.unique(dftrain['ID'].values)
dftrain.index.shape

(84387,)

In [438]:
weightdict={'Author':0.4,'Title':0.5,'Publisher':0.1}
booksim = book_similarity_vec(dftrain_books[dftrain_books['Title']=='신의 가면 1'],dftrain_books,weightdict=weightdict)


In [440]:
booksim.sort(['similarity'],ascending=False)

,index,Author,Title,Publisher,similarity
10,10,조지프 캠벨 저/이진구 역,신의 가면 1,까치(까치글방),0.000050
5,5,조지프 캠벨,신의 가면 3,까치(까치글방),0.000049
6,6,조지프 캠벨,신의 가면 2,까치(까치글방),0.000049
7,7,조지프 캠벨 저/정영목 역,신의 가면 4,까치(까치글방),0.000049
9690,26477,<조지프 캠벨> 저/<다이앤 K. 오스본> 편/<박중서> 역,신화와 인생,갈라파고스,0.000028
9,9,<조지프 캠벨> 저/<노혜숙> 역,"블리스, 내 인생의 신화를 찾아서",아니마,0.000025
671,857,<송지나> 저,신의 1,비채,0.000017
17640,68721,<그레이엄 헨콕> 저,신의 암호 (상),까치(까치글방),0.000017
17641,68722,<그레이엄 헨콕> 저,신의 암호 (하),까치(까치글방),0.000017
17388,66994,<이두석> 저,붓다의 가르침 1,맑은소리 맑은나라,0.000017


In [454]:
def predict_percentage(dftrain,dftest,Nrecbooks,weightdict):
    users = np.unique(dftrain['ID'].values)
    totsuccess = 0
    dftrain_books = dftrain[['Author','Title','Publisher','ISBN']]
    dftrain_books = dftrain_books[~dftrain_books.duplicated()]
    dftrain_books = dftrain_books.reset_index()
    for i,user in enumerate(users):
        dftestuser = dftest[dftest['ID']==user]    
        
        dfbooksim = book_similarity_vec(dftestuser,dftrain_books,weightdict=weightdict)
        
        for book in dftestuser['ISBN']:
            dfbooksim = dfbooksim[dfbooksim['ISBN']!=book]
        
        tempdf = dfbooksim.sort(['similarity'],ascending=False)
        ISBNrecbooks = tempdf['ISBN'].values[1:Nrecbooks+1]
        successarr = np.array([isbn in ISBNrecbooks for isbn in dftestuser['ISBN'].values])
        totsuccess = totsuccess + np.sum(successarr)
    print i
            
    return totsuccess/(Nrecbooks*users.shape[0])
            
        

In [455]:
predict_percentage(dftrain,dfval,5,weightdict)

KeyboardInterrupt: 